In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from dataset import Dataset
from features import StrategiesSignals

# Download data from binance

In [2]:
TICKER = "BTCUSDT"
data_binance = Dataset()
data = data_binance.get_data(days=90, ticker=TICKER, ts='5m')

In [3]:
data.tail(5)

,open,high,low,close,volume
time,,,,,
2021-08-28 23:45:00,48797.96,48822.58,48760.41,48822.58,41.52722
2021-08-28 23:50:00,48822.57,48880.00,48818.23,48852.20,37.69213
2021-08-28 23:55:00,48850.00,48879.98,48834.23,48840.32,30.06146
2021-08-29 00:00:00,48840.33,48895.36,48836.51,48895.35,42.17464
2021-08-29 00:05:00,48895.35,48926.68,48840.00,48846.35,158.98450


# Create target variables

In [53]:
size_body = 0.7
def find_candle(candle, size_body):
    rule_1 = (candle.get('close') - candle.get('low')) / (.001 + candle.get('high') - candle.get('low'))
    rule_2 = (candle.get('open') - candle.get('low')) / (.001 + candle.get('high') - candle.get('low'))
    rule_3 = candle.get('close') > candle.get('open')
    
    # Define rules
    bar = (
        ((candle.get('high') - candle.get('low')) > 3*(candle.get('open') - candle.get('close'))) &
        ((rule_1 > size_body) & (rule_1 < 1)) &
        ((rule_2 > size_body) & (rule_2 < 1)) &
        (rule_3))
    return 1 if bar else 0


In [19]:
rule_1 = isinstance('tes_bar', str)
rule_2 = isinstance('big_volume', str)
rule_3 = isinstance('corretction 1%', str)
signal = rule_1 & rule_2 &rule_3
print(1 if signal else 0)


1


In [46]:
# Look for the biggest volume for the last 1 hour (12 bars)
data['rolling_volume'] = data['volume'].rolling(12).max()

In [47]:
data[['volume', 'rolling_volume']].tail(10)

,volume,rolling_volume
time,,
2021-08-28 23:20:00,50.73259,122.01193
2021-08-28 23:25:00,37.73668,122.01193
2021-08-28 23:30:00,34.03604,122.01193
2021-08-28 23:35:00,32.27880,87.17731
2021-08-28 23:40:00,46.69198,65.04374
2021-08-28 23:45:00,41.52722,65.04374
2021-08-28 23:50:00,37.69213,65.04374
2021-08-28 23:55:00,30.06146,58.61879
2021-08-29 00:00:00,42.17464,58.61879


In [55]:
data['signal'] = 0
for index, value in data.iterrows():
    data.loc[index, 'signal'] = find_candle(value, size_body)    
    data.loc[index, 'signal_volume'] = 1 if value.get('volume') == value.get('rolling_volume') else 0

In [56]:
# data.groupby('signal').count()
data.groupby(['signal', 'signal_volume']).count()

open   high    low  close  volume  rolling_volume  \
signal signal_volume                                                       
0      0.0            22982  22982  22982  22982   22982           22971   
       1.0             2326   2326   2326   2326    2326            2326   
1      0.0              512    512    512    512     512             512   
       1.0               47     47     47     47      47              47   

                      lag_1  lag_2  lag_3  lag_4  ...  lag_10  lag_11  lag_12  \
signal signal_volume                              ...                           
0      0.0            22982  22981  22980  22979  ...   22973   22972   22971   
       1.0             2325   2325   2325   2325  ...    2325    2325    2325   
1      0.0              512    512    512    512  ...     512     512     512   
       1.0               47     47     47     47  ...      47      47      47   

                      lag_13  lag_14  lag_15  lag_16  lag_17  lag_18  lag_19  
signal signal_volume                                                          
0      0.0             22970   22969   22968   22967   22966   22965   22964  
       1.0              2325    2325    2325    2325    2325    2325    2325  
1      0.0               512     512     512     512     512     512     512  
       1.0                47      47      47      47      47      47      47  

[4 rows x 25 columns]

In [57]:

# create lags
lags = range(1, 20)

# run loop to calculate target percent change
for i in lags:
    data[f'lag_{i}'] = data['close'].pct_change(-i)

# generate names of colimns
name_lags = [f'lag_{i}' for i in lags]
# Append our signals
name_lags.append('signal')
name_lags.append('signal_volume')
data[name_lags].groupby(['signal', 'signal_volume']).median().T #.plot()

signal                0                   1          
signal_volume       0.0       1.0       0.0       1.0
lag_1         -0.000002  0.000020  0.000052 -0.000155
lag_2         -0.000005 -0.000051  0.000073  0.000381
lag_3         -0.000055  0.000005  0.000191  0.001026
lag_4         -0.000075  0.000055 -0.000061  0.000737
lag_5         -0.000094  0.000107  0.000386  0.000906
lag_6         -0.000171  0.000076  0.000281  0.001234
lag_7         -0.000170  0.000138  0.000364  0.001097
lag_8         -0.000207  0.000003  0.000431 -0.000632
lag_9         -0.000230 -0.000040  0.000341 -0.000412
lag_10        -0.000265 -0.000084  0.000370  0.000173
lag_11        -0.000269 -0.000105  0.000349 -0.000504
lag_12        -0.000298 -0.000152  0.000305  0.000305
lag_13        -0.000326 -0.000101  0.000475  0.000688
lag_14        -0.000304  0.000075  0.000542  0.001026
lag_15        -0.000291 -0.000032  0.000456  0.000645
lag_16        -0.000323 -0.000112  0.000241  0.001319
lag_17        -0.000358 -0.000207  0.000727  0.001409
lag_18        -0.000333 -0.000148  0.000952  0.001529
lag_19        -0.000352 -0.000176  0.001034  0.000353

In [58]:
# Returns 8% for 90 days
0.000529 * 47

0.024862999999999996

In [44]:
# Find the best leverage on binance featires for BTCUSDT
# Biggest drowdown 4%
# The best leverage is 100 / 4 = 25. More safe 20 leverage
# Expecting returns: 8% * 20 = 160% for 90 says

data[['lag_18', 'signal']].groupby('signal').min()

,lag_18
signal,
0,-0.100473
1,-0.039730


In [45]:
100 / 4

25.0

In [39]:
data[['lag_18', 'signal']].groupby('signal').count()

,lag_18
signal,
0,25614
1,235


In [58]:
data[['lag_18', 'signal']].groupby('signal').min()

,lag_18
signal,
0,-0.100473
1,-0.039730
